In [1]:
# Some tests of NuSTAR lightcurves in comparison to other X-ray data
# 
# 11-04-2023    IGH
# 17-04-2023    Add in per NuSTAR detector quadrant panel
#               Added in more events
# 21-04-2023    Added in AIA HEK flares

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy import timeseries as ts
from sunpy.time import parse_time
from astropy.io import fits
import astropy.units as u
import datetime
import pandas as pd
import os
from pathlib import Path
import glob
from stixdcpy.quicklook import LightCurves
from stixdcpy.net import Request as jreq
from stixdcpy import auxiliary as aux

import nsx_func

import warnings
warnings.simplefilter('ignore')

plt.rcParams.update({'font.size': 18,'font.family':"sans-serif",\
            'font.sans-serif':"Arial",'mathtext.default':"regular"})

In [3]:
# List of all the NuSTAR observations, with my naming, 
# based off first day of observing per campaign
dobs=['20140910','20141101','20141211',
    '20150429','20150901',
    '20160219','20160422','20160726',
    '20170321','20170821','20170911','20171010',
    '20180529','20180907','20180928',
    '20190112','20190412','20190425','20190702',
    '20200129','20200221','20200606','20200912',
    '20210108','20210429','20210720','20210730','20211117',
    '20220224','20220603','20220906','20221209',
    '20230318']

# Data directory
# ddir=str(Path.home())+'/data/heasarc_nustar/'
ddir='/Volumes/Samsung_T5/data/heasarc_nustar/'

In [4]:
#  sunpy still only returns the 1s GOES16-18 data and not the avg1min one
# So will need to manually download myself, and then search in that local directory for them
#  Just work with GOES16 as covers 2020 to now
# i.e. https://ngdc.noaa.gov/stp/satellite/goes/doc/GOES_XRS_readme.pdf
# 
# Need to manually get the data from 
# https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes16/l2/data/xrsf-l2-avg1m_science/
gdir=str(Path.home())+'/data/goes_xrs/'

In [5]:
# Do all of 2022 and 2023 (so far) but not June (as mosaics and don't have locally)
gdir=str(Path.home())+'/data/goes_xrs/'

for icamp in [21,23,24,25,26,27,28,29,30,31,32]:
# for icamp in [28]:
    maindir=ddir+'ns_'+dobs[icamp]+'/'
    print(icamp, ' --- ',maindir)
    # Most start with 20 or 80 or 90?
    ids = [f.name for f in os.scandir(maindir) \
        if (f.is_dir() and (f.name.startswith('20') or f.name.startswith('80') or f.name.startswith('90')))]
    ids=sorted(ids)
    # Filter out random other dirs
    ids=[f for f in ids if not f.endswith('_new')]
    # Remove the mosiac from 20220603
    if (dobs[icamp] == '20220603'):
        ids=[f for f in ids if f.startswith('2062400')]      
    print("#: ",len(ids),' --- ',ids)

    # # STIX + GOES/XRS + NuSTAR  lightcurves
    for nsid in ids:
        print(nsid)

        df_test=nsx_func.nsrate(maindir=maindir,nsid=nsid,lvt=True,englow=2,enghigh=10)
        df10 = df_test.resample('10s', level=0).mean()
        lla=df10["lvta"].values
        lla=lla[lla>0]
        llb=df10["lvtb"].values
        llb=llb[llb>0]
        lvta=f'({100*np.min(lla):.2f} - {100*np.max(lla):.2f}%)'
        lvtb=f'({100*np.min(llb):.2f} - {100*np.max(llb):.2f}%)'
        # yrange for NuSTAR plot
        alldata=np.concatenate((df10["rta"].values,df10["rtb"].values))
        alldata=alldata[alldata > 0]
        maxy=1.4*max(alldata)
        miny=0.98*min(alldata)
        yr=[miny,maxy]
        #  Get it per detector quadrant
        dft0=nsx_func.nsrate(maindir=maindir,nsid=nsid,lvt=False,englow=2,enghigh=10,det_id=[0])
        df0= dft0.resample('10s', level=0).mean()
        dft1=nsx_func.nsrate(maindir=maindir,nsid=nsid,lvt=False,englow=2,enghigh=10,det_id=[1])
        df1= dft1.resample('10s', level=0).mean()
        dft2=nsx_func.nsrate(maindir=maindir,nsid=nsid,lvt=False,englow=2,enghigh=10,det_id=[2])
        df2= dft2.resample('10s', level=0).mean()
        dft3=nsx_func.nsrate(maindir=maindir,nsid=nsid,lvt=False,englow=2,enghigh=10,det_id=[3])
        df3= dft3.resample('10s', level=0).mean()
        alldet=np.concatenate((df0["rta"].values,df1["rta"].values,df2["rta"].values,df3["rta"].values))
        yrdet=[0.98*min(alldet),1.3*max(alldet)]

        # Time range to nearest 5mins
        mint=df10.index[0].to_pydatetime()
        mint-=datetime.timedelta(minutes=5,seconds=mint.second)
        mint-=datetime.timedelta(minutes=mint.minute % 5)
        maxt=df10.index[-1].to_pydatetime()
        maxt+=datetime.timedelta(minutes=5,seconds=60-maxt.second)
        maxt-=datetime.timedelta(minutes=maxt.minute % 5)
        tr=[mint,maxt]

        # Get the GOES XRS data
        gtimf=np.unique([f'{mint:%Y%m%d}',f'{maxt:%Y%m%d}'])
        gfiles=[glob.glob(gdir+'*'+g+'*')[0] for g in gtimf]
        trange=a.Time(tr[0],tr[1])
        gxrs = ts.TimeSeries(gfiles, concatenate=True)
        gxrst=gxrs.truncate(trange.start.iso,trange.end.iso)
        tg_tims=gxrst.index
        tg_x05=gxrst.quantity("xrsa").value
        tg_x18=gxrst.quantity("xrsb").value

        # Get the STIX quicklook data
        slc = LightCurves.from_sdc(start_utc=tr[0], end_utc=tr[1], ltc=True)
        # Only plot if STIX data there
        if not slc.data is None:
            stx_eb=slc.data["energy_bins"]
            # Connvert to dataframe, then can easily truncate to actual timerange
            slc_df = pd.DataFrame(np.array(slc.data["counts"]).T,index=slc.time, columns=slc.data["energy_bins"]["names"])
            slc_df_tr = slc_df.truncate(tr[0],tr[1])
            slcdata=slc_df_tr[stx_eb["names"][0]].values
        
        # Where is STIX?
        emph=aux.Ephemeris.from_sdc(start_utc=tr[0], end_utc=tr[1], steps=1)

        # Anything in the GOES flare list? Obviously needs the internet to work
        gsflin = Fido.search(trange,a.hek.EventType("FL"),a.hek.FRM.Name == "SSW Latest Events")
        gsflhk=gsflin["hek"]
        # Anything from AIA?
        aiaflin = Fido.search(trange,a.hek.EventType("FL"),a.hek.FRM.Name =="Flare Detective - Trigger Module")
        aiaflhk=aiaflin["hek"]  

        # # Stuff in STIX flare list?
        # # Don't fully trust what this is giving me (and no positions of course)
        # stxfl=jreq.fetch_flare_list(trange.start.isot, trange.end.isot, sort='time')

        # Control the time axis labelling
        myFmt = matplotlib.dates.DateFormatter('%H:%M')
        majorx= matplotlib.dates.MinuteLocator(interval=10)
        minorx= matplotlib.dates.MinuteLocator(interval=1)
        plt.rcParams.update({'font.size': 16})
        fig, axs= plt.subplots(4,figsize=(10, 12))
        
        if not slc.data is None:
            axs[0].plot(slc_df_tr.index, slcdata,drawstyle='steps-post',marker=None,color='steelblue',lw=2,\
                label=f'STIX {stx_eb["names"][0]} (D$_S$: {emph.data["orbit"]["sun_solo_r"][0]:.2f}, ESS: {emph.data["orbit"]["earth_sun_solo_angle"][0]:.2f})')
            axs[0].set_ylabel('STIX [counts]')
            axs[0].set_ylim([0.98*np.min(slcdata),1.3*np.max(slcdata)])
            axs[0].legend(loc=2,fontsize=14)
        else:
            axs[0].plot(tr, [10,10],drawstyle='steps-post',marker=None,color='steelblue',lw=2,\
                label=f'STIX No Data? (D$_S$: {emph.data["orbit"]["sun_solo_r"][0]:.2f}, ESS: {emph.data["orbit"]["earth_sun_solo_angle"][0]:.2f})')
            axs[0].set_ylabel('STIX [counts]')
            axs[0].set_ylim([20,200])
            axs[0].legend(loc=2,fontsize=14)

        # for sf in stxfl:
        #     ptim_shft=parse_time(sf.get('peak_UTC'))+slc.data["SOLO_EARTH_LIGHT_TIME"]*u.s
        #     axs[0].axvline(ptim_shft.iso,color='steelblue',ls='--')

        axs[1].plot(tg_tims,tg_x18,drawstyle='steps-post',\
            marker=None,color='firebrick',lw=2,label='G16 $1-8\;\AA$')
        axs[1].set_ylabel('XRS [W$\;m^{-2}$]')
        # Only set limits if not all missing/nans
        if not np.isnan(np.nanmin(tg_x18)):
            yrgs=[0.95*np.nanmin(tg_x18),1.25*np.nanmax(tg_x18)]
        else:
            yrgs=[0.5e-7,3.5e-7]
        mngs=np.mean(yrgs)
        gspow = int(f"{mngs:e}".split('e')[1])
        mngs=round(mngs/10**gspow)*10**gspow
        match gspow:
            case -8:
                gscls="A"
            case -7:
                gscls="B"
            case -6:
                gscls="C"
            case -5:
                gscls="M"
        if (mngs < yrgs[0]):   
            yrgs[0] = mngs
        if (mngs > yrgs[1]):   
            yrgs[1] = mngs
        axs[1].set_ylim(yrgs)
        axs[1].plot(tr,[mngs,mngs],linestyle='dotted',color='grey')
        axs[1].annotate(f' {gscls}{mngs/10**gspow:.0f}',(tr[1],mngs),
                        color='grey',fontweight="bold")

        for gg in gsflhk:
            tfl=a.Time(gg["event_starttime"],gg["event_endtime"])
            ptim=parse_time(gg["event_peaktime"]).datetime
            axs[1].axvspan(tfl.start.datetime,tfl.end.datetime,color='silver',alpha=0.4, \
                label=gg["fl_goescls"]+f' ({ptim:%H:%M})')
            # for ax in axs:
            #     ax.axvline(ptim,color='black',ls='-.')
            axs[1].axvline(ptim,color='black',ls='-.')
            axs[2].axvline(ptim,color='black',ls='-.')
            axs[3].axvline(ptim,color='black',ls='-.')
        for aa in aiaflhk:
            #  Only plot it if peak time during NuSTAR time window
            if (aa["event_peaktime"] >= tr[0]) and (aa["event_peaktime"] <= tr[1]):
                ptim=parse_time(aa["event_peaktime"]).datetime
                axs[1].axvline(ptim,color='darkorchid',ls=':')
                # axs[1].axvline(ptim,color='darkorchid',ls=':',label=f'AF ({ptim:%H:%M})')
                axs[2].axvline(ptim,color='darkorchid',ls=':')
                axs[3].axvline(ptim,color='darkorchid',ls=':')
        axs[1].legend(loc=2,ncols=5,fontsize=14)

        axs[2].plot(df0.index,df0["rta"].values,drawstyle='steps-post',lw=2,color='royalblue',label='A0')
        axs[2].plot(df1.index,df1["rta"].values,drawstyle='steps-post',lw=2,color='darkorange',label='A1')
        axs[2].plot(df2.index,df2["rta"].values,drawstyle='steps-post',lw=2,color='lightseagreen',label='A2')
        axs[2].plot(df3.index,df3["rta"].values,drawstyle='steps-post',lw=2,color='orchid',label='A3')

        axs[2].set_ylabel('NuSTAR [count$\;s^{-1}$]')
        axs[2].set_ylim(yrdet)
        axs[2].legend(loc=2,ncols=4,fontsize=14)

        axs[3].plot(df10.index,df10["rta"].values,drawstyle='steps-post',\
            lw=2,color='rebeccapurple',label='A >2keV '+lvta)
        axs[3].plot(df10.index,df10["rtb"].values,drawstyle='steps-post',\
            lw=2,color='teal',label='B >2keV '+lvtb)
        axs[3].set_xlabel(f'Start Time {mint:%d-%b-%Y %H:%M:%S}')
        axs[3].set_ylabel('NuSTAR [count$\;s^{-1}$]')
        axs[3].set_ylim(yr)
        axs[3].legend(loc=2,fontsize=14,ncols=2)
        for ax in axs:
            ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0), useOffset=True)
            ax.set_xlim(tr)
            ax.xaxis.set_major_locator(majorx)
            ax.xaxis.set_minor_locator(minorx)
            ax.xaxis.set_major_formatter(myFmt)
        plt.annotate(nsid,(0.01,0.01),xycoords='figure fraction',fontsize=14)

        fig.subplots_adjust(bottom=0.06, top=0.97, left=0.12, right=0.95)
        plt.savefig('figs/ns_'+dobs[icamp]+f'/ltc_{mint:%Y%m%d_%H%M}_'+nsid+'_ngs.png')
        # as looping don't need to plot the figures to the notebook
        plt.close()
        # break
    break


28  ---  /Volumes/Samsung_T5/data/heasarc_nustar/ns_20220224/
#:  12  ---  ['20621001001', '20621002001', '20621003001', '20621004001', '20622001001', '20622002001', '20622003001', '20622004001', '20623001001', '20623002001', '20623003001', '20623004001']
20621001001
